In [ ]:
from pathlib import Path
import json
import subprocess
import shlex

# Step 1: Clean cache

What I did was to set a new conan cache

```shell
mkdir ~/.conan/bump_deps
conan config set storage.path="./bump_deps" 
```

Then I cmake-configured a Release and a Debug build on the M1 mac (so it'll pick up most of build dependencies too since packages aren't readily available)

# Supporting Classes

In [ ]:
class PkgInfo:
    
    @staticmethod
    def from_metadata(metadata_path):
        
        name, version, user, channel = p.relative_to(CONAN_CACHE).parent.parts
        
        with open(p, 'r') as f:
            data = json.load(f)
            revision = data['recipe']['revision']
            
        return PkgInfo(name=name, version=version, user=user, channel=channel, revision=revision)

    def __init__(self, name, version, user, channel, revision):
        self.name = name
        self.version = version
        self.user = None
        if user is not None and user != '_':
            self.user = user
        self.channel = None
        if channel is not None and channel != '_':
            self.channel = channel
            
        self.revision = revision

        self.remote = 'conancenter'
        if self.name == 'ruby_installer':
            self.remote = 'bincrafters'
        elif self.name == 'openstudio_ruby':
            self.remote = 'nrel'
            
        
    def search_packages(self):
        """Filters out packages (such as Windows MSVC 15)
        
        TODO: improvement: remove recipes with shared=True?
        """
        json_p = Path(f'{self.name}.json')
        subprocess.check_call(shlex.split(f"conan search -r {self.remote} {self.reference()} --json {json_p}"),
                              stderr=subprocess.DEVNULL, stdout=subprocess.DEVNULL)
        with open(json_p, 'r') as f:
            data = json.load(f)
        json_p.unlink(missing_ok=False) # remove tmp json
        
        packages = data['results'][0]['items'][0]['packages']
        keep_packages = []
        skipped_packages = []
        for p in packages:
            settings = p['settings']
            os_ = settings.get('os', None)
            compiler_version = settings.get('compiler.version', None)
            compiler = settings.get('compiler', None)
            libcxx  = settings.get('compiler.libcxx', None)
            
            if os_ == 'Windows':
                if compiler_version not in ['16', '17', None]:
                    print(f"Skipping Windows {compiler_version=} for pkg {self.name}")
                    skipped_packages.append(p)
                    continue
                
                runtime = settings.get('compiler.runtime', None)
                if runtime not in ['MD', 'MDd', None]:
                    print(f"Skipping Windows {runtime=} for pkg {self.name}")
                    skipped_packages.append(p)
                    continue
            elif os_ == 'Linux':
                
                if compiler not in ['gcc', 'clang', None]:
                    print(f"Skipping Linux {compiler=} for pkg {self.name}")
                    skipped_packages.append(p)
                    continue
                    
                if libcxx not in ['libstdc++11', 'libc++', None]:
                    print(f"Skipping Linux {libcxx=} for pkg {self.name} with ({compiler=})")
                    skipped_packages.append(p)
                    continue
                    
                if compiler == 'gcc':
                    if compiler_version not in ['7', '8', '9', '10', '11', '12', None]:
                        print(f"Skipping Linux gcc {compiler_version=} for pkg {self.name}")
                
            elif os_ == 'Macos':
                if libcxx not in ['libc++', None]:
                    print(f"Skipping Macos {libcxx=} for pkg {self.name} with ({compiler=})")
                    skipped_packages.append(p)
                    continue
            elif os_ is None:
                pass
            else:
                print("Unknown os: {os_}")
                skipped_packages.append(p)
                continue
                
            keep_packages.append(p)
            
        return keep_packages, skipped_packages

    def download_all(self):
        subprocess.check_call(
            ['conan', 'download', '-r', self.remote, self.reference()],
            # stderr=subprocess.DEVNULL, stdout=subprocess.DEVNULL
        )
        
    def download_specific_packages(self):
        """Filters out the stuff we don't need by calling `search_packages`
        """
        packages, _ = self.search_packages()
        
        for p_dict in packages:
            print(p_dict)
            pkg_id = p_dict['id']
            subprocess.check_call(
                ['conan', 'download', '-r', self.remote, f"{self.reference()}:{pkg_id}"],
                #stderr=subprocess.DEVNULL, stdout=subprocess.DEVNULL
            )
        
    def upload_to_nrel(self):
        subprocess.check_call(
            ['conan', 'upload', '-r', 'nrel', '--all', '--parallel',
             '--no-overwrite', 'all', self.reference()],
            # stderr=subprocess.DEVNULL, stdout=subprocess.DEVNULL
        )
        
    def package_dir(self):
        p = CONAN_CACHE / f"{self.name}/{self.version}"
        if self.user is not None:
            p /= f"{self.user}/{self.channel}"
        else:
            p /= "_/_"
        p /= 'package'
        return p
    
    def cleanup_skipped_packages(self, remote=None):
        """if remote is none, cleans up your local cache"""
        _, skipped_packages = self.search_packages()
        for p_dict in skipped_packages:
            pkg_id = p_dict['id']
            cmd_args = ['conan', 'remove', '-f', self.reference(), '-p', pkg_id]
            if remote is not None:
                cmd_args += ['-r', remote]
                
            subprocess.run(cmd_args)
    
    def reference(self):
        s = f"{self.name}/{self.version}@"
        if self.user is not None:
            s += f"{self.user}/{self.channel}"
        s += f"#{self.revision}"
        return s
    
    def __repr__(self):
        return self.reference()
    
    def __eq__(self, other):
        return self.reference() == other.reference()

# Download packages

## Parse my local cache

In [ ]:
CONAN_CACHE = Path('/Users/julien/.conan/bump_deps/')

pkg_infos = []
for p in CONAN_CACHE.glob('**/metadata.json'):
    pkg_infos.append(PkgInfo.from_metadata(p))
pkg_infos.sort(key=lambda p: p.name)

In [ ]:
len(pkg_infos)

## Compare packages I have in my cache with the conan.lock produced when building OS

In [ ]:
conan_lock = '/Users/julien/Software/Others/OS-build-bump/conan.lock'
with open(conan_lock, 'r') as f:
    conan_lock_data = json.load(f)

In [ ]:
conan_lock_data.keys()

In [ ]:
pkg_infos_lock = []
for k, node in conan_lock_data['graph_lock']['nodes'].items():
    if not 'ref' in node:
        print(f"{k=} has no ref (it's node 0, that's normal)")
        continue
    n, revision = node['ref'].split('#')
    split = n.split('/')
    if len(split) == 2:
        name, version = split
        user, channel = (None, None)
    elif len(split) == 3:
        name, version_user, channel = split
        version, user = version_user.split('@')
    else:
        raise ValueError(f"Wrong size: {split}")
    pkg_infos_lock.append(PkgInfo(name=name, version=version, user=user, channel=channel, revision=revision))

In [ ]:
pkg_infos_lock

In [ ]:
set(x.reference() for x in pkg_infos) - set(x.reference() for x in pkg_infos_lock)

In [ ]:
set(x.reference() for x in pkg_infos_lock) - set(x.reference() for x in pkg_infos)

## Download all packages

In [ ]:
for pkg_info in pkg_infos:
    if pkg_info.name == 'openstudio_ruby':
        continue
        
    # if pkg_info.name != 'ruby_installer':
    #     continue
    print(pkg_info.name)
    #keep_packages, skip_packages = pkg_info.search_packages()
    # pkg_info.download_specific_packages()
    
    # Download all has the benefit of running in parallel... so it's faster
    # We'll clean it up later
    pkg_info.download_all()
    print("\n")

## Clean up my local cache to remove unwanted packages

In [ ]:
for pkg_info in pkg_infos:
    if pkg_info.name == 'openstudio_ruby':
        continue
        
    if pkg_info.name != 'boost':
         continue
    print(pkg_info.name)
    #keep_packages, skip_packages = pkg_info.search_packages()
    pkg_info.cleanup_skipped_packages()
    print("\n")

# Upload to NREL

In [ ]:
for pkg_info in pkg_infos:
    if pkg_info.name == 'openstudio_ruby':
        continue
    print(pkg_info.name)
    #keep_packages, skip_packages = pkg_info.search_packages()
    pkg_info.upload_to_nrel()
    print("\n")

# In one go